### 1. Import libraries, connect to SQL database, and convert dataframe to SQL table

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import pandas as pd
import sqlite3

In [33]:
# import xlsx file
file_name = "nashville_housing_data.xlsx"
my_sheet_name = "Sheet1"

df = pd.read_excel(file_name, sheet_name = my_sheet_name, index_col=0)

In [34]:
df.head()

,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
UniqueID,,,,,,,,,,,,,,,,,,
2045,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE","1808 FOX CHASE DR, GOODLETTSVILLE, TN",2.3,GENERAL SERVICES DISTRICT,50000.0,168200.0,235700.0,1986.0,3.0,3.0,0.0
16918,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE","1832 FOX CHASE DR, GOODLETTSVILLE, TN",3.5,GENERAL SERVICES DISTRICT,50000.0,264100.0,319000.0,1998.0,3.0,3.0,2.0
54582,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE","1864 FOX CHASE DR, GOODLETTSVILLE, TN",2.9,GENERAL SERVICES DISTRICT,50000.0,216200.0,298000.0,1987.0,4.0,3.0,0.0
43070,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",2016-01-29,255000,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.","1853 FOX CHASE DR, GOODLETTSVILLE, TN",2.6,GENERAL SERVICES DISTRICT,50000.0,147300.0,197300.0,1985.0,3.0,3.0,0.0
22714,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",2014-10-10,278000,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.","1829 FOX CHASE DR, GOODLETTSVILLE, TN",2.0,GENERAL SERVICES DISTRICT,50000.0,152300.0,202300.0,1984.0,4.0,3.0,0.0


In [35]:
# connect to SQL database
conn = sqlite3.connect('my_projects.db')

df.to_sql("nashville_housing", conn, if_exists='replace')

/home/tianyuan/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [36]:
%load_ext sql

%sql sqlite:///my_projects.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### 2. Populate null values

#### 2.1 check null values in "PropertyAddress" column

In [37]:
%%sql

SELECT
    *
From
    nashville_housing
WHERE
    PropertyAddress IS NULL;

 * sqlite:///my_projects.db
Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
43076,025 07 0 031.00,SINGLE FAMILY,None,2016-01-15 00:00:00,179900,20160120-0005776,No,"COSTNER, FRED & CAROLYN","410 ROSEHILL CT, GOODLETTSVILLE, TN",0.96,CITY OF GOODLETTSVILLE,30000.0,70000.0,100000.0,1964.0,3.0,1.0,0.0
39432,026 01 0 069.00,VACANT RESIDENTIAL LAND,None,2015-10-23 00:00:00,153000,20151028-0109602,No,"SHACKLEFORD, MICHAEL C., JR.","141 TWO MILE PIKE, GOODLETTSVILLE, TN",0.17,CITY OF GOODLETTSVILLE,21100.0,121600.0,142700.0,2015.0,3.0,2.0,0.0
45290,026 05 0 017.00,SINGLE FAMILY,None,2016-03-29 00:00:00,155000,20160330-0029941,No,"TRIPP, MARVIN S. & DEBORAH YOUNG","208 EAST AVE, GOODLETTSVILLE, TN",0.2,CITY OF GOODLETTSVILLE,21100.0,130200.0,151300.0,2008.0,3.0,2.0,0.0
53147,026 06 0A 038.00,RESIDENTIAL CONDO,None,2016-08-25 00:00:00,144900,20160831-0091567,No,None,None,None,None,None,None,None,None,None,None,None
43080,033 06 0 041.00,SINGLE FAMILY,None,2016-01-04 00:00:00,170000,20160107-0001526,No,"FRANK, ZACHARY & NIKI","1129 CAMPBELL RD, GOODLETTSVILLE, TN",0.24,GENERAL SERVICES DISTRICT,35000.0,110500.0,145500.0,2000.0,3.0,2.0,0.0
45295,033 06 0A 002.00,SINGLE FAMILY,None,2016-03-29 00:00:00,210000,20160331-0030709,No,None,None,None,None,None,None,None,None,None,None,None
48731,033 15 0 123.00,SINGLE FAMILY,None,2016-05-05 00:00:00,199900,20160506-0045368,No,"COLEMAN, AARON A. & CECIL, CORRIE J.","438 W CAMPBELL RD, GOODLETTSVILLE, TN",1.39,GENERAL SERVICES DISTRICT,45000.0,90300.0,135300.0,1954.0,2.0,1.0,0.0
36531,034 03 0 059.00,SINGLE FAMILY,None,2015-08-13 00:00:00,245000,20150819-0083759,No,"DILICK, JOHN MARK & ANNETTE A.","2117 PAULA DR, MADISON, TN",1.01,GENERAL SERVICES DISTRICT,32000.0,170000.0,228300.0,1964.0,4.0,3.0,0.0
46919,034 07 0B 015.00,VACANT RESIDENTIAL LAND,None,2016-04-27 00:00:00,40000,20160304-0020905,Yes,None,None,None,None,None,None,None,None,None,None,None
44264,034 16 0A 004.00,VACANT RESIDENTIAL LAND,None,2016-02-04 00:00:00,130000,20160205-0011327,Yes,None,None,None,None,None,None,None,None,None,None,None


#### 2.2 Use ***ParcelID*** to reference property addresses in self-join

In [38]:
%%sql

SELECT
    t1.ParcelID,
    t1.PropertyAddress,
    t2.ParcelID,
    t2.PropertyAddress
FROM
    nashville_housing AS t1
JOIN
    nashville_housing AS t2
ON
    t1.ParcelID = t2.ParcelID AND t1.[UniqueID ] <> t2.[UniqueID ]
WHERE
    t1.PropertyAddress IS NULL;

 * sqlite:///my_projects.db
Done.


ParcelID,PropertyAddress,ParcelID_1,PropertyAddress_1
025 07 0 031.00,None,025 07 0 031.00,"410 ROSEHILL CT, GOODLETTSVILLE"
026 01 0 069.00,None,026 01 0 069.00,"141 TWO MILE PIKE, GOODLETTSVILLE"
026 05 0 017.00,None,026 05 0 017.00,"208 EAST AVE, GOODLETTSVILLE"
026 06 0A 038.00,None,026 06 0A 038.00,"109 CANTON CT, GOODLETTSVILLE"
033 06 0 041.00,None,033 06 0 041.00,"1129 CAMPBELL RD, GOODLETTSVILLE"
033 06 0A 002.00,None,033 06 0A 002.00,"1116 CAMPBELL RD, GOODLETTSVILLE"
033 15 0 123.00,None,033 15 0 123.00,"438 W CAMPBELL RD, GOODLETTSVILLE"
034 03 0 059.00,None,034 03 0 059.00,"2117 PAULA DR, MADISON"
034 03 0 059.00,None,034 03 0 059.00,"2117 PAULA DR, MADISON"
034 07 0B 015.00,None,034 07 0B 015.00,"2524 VAL MARIE DR, MADISON"


#### 2.3 Populate null values in "PropertyAddress" column referenced to according ***ParcelID***

In [39]:
%%sql

UPDATE
    nashville_housing
SET
    PropertyAddress = (SELECT 
                           nh.PropertyAddress
                       FROM 
                           nashville_housing AS nh
                       WHERE 
                           nashville_housing.ParcelID = nh.ParcelID)
WHERE EXISTS (SELECT 
                  nh.PropertyAddress
              FROM 
                  nashville_housing AS nh
              WHERE 
                  nashville_housing.ParcelID = nh.ParcelID
                  AND nashville_housing.[UniqueID ] <> nh.[UniqueID ]
                  AND nashville_housing.PropertyAddress IS NULL)

 * sqlite:///my_projects.db
29 rows affected.


[]

### 3. Standardize data formats

#### 3.1 convert "SaleDate" column from datetime to date

In [40]:
%%sql

UPDATE
    nashville_housing
SET
    SaleDate = DATE(SaleDate);

 * sqlite:///my_projects.db
56477 rows affected.


[]

#### 3.2 Split "PropertyAddress" column into individual columns

- First, parse property address out of "PropertyAddress" column and set values into a new column "SplitPropertyAddress"

In [41]:
%%sql

ALTER TABLE
    nashville_housing
ADD COLUMN
    SplitPropertyAddress TEXT(255);

UPDATE
    nashville_housing
SET
    SplitPropertyAddress = SUBSTR(PropertyAddress, 1, INSTR(PropertyAddress, ", ")-1);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

- Then, parse city information out of "PropertyAddress" column and set values into a new column "SplitPropertyCity"

In [42]:
%%sql

ALTER TABLE
    nashville_housing
ADD COLUMN
    SplitPropertyCity TEXT(255);

UPDATE
    nashville_housing
SET
    SplitPropertyCity = SUBSTR(PropertyAddress, INSTR(PropertyAddress, ", ")+2);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

#### 3.3 Split "OwnerAddress" column into individual columns for address, city, and state

- First, parse only property address out of "OwnerAddress" column and set values into a new column "SplitOwnerAddress"

In [43]:
%%sql

ALTER TABLE
    nashville_housing
ADD COLUMN
    SplitOwnerAddress TEXT(255);

UPDATE
    nashville_housing
SET
    SplitOwnerAddress = SUBSTR(OwnerAddress, 1, INSTR(OwnerAddress, ", ")-1);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

- Second, reset "OwnerAddress" column to only owners' city and state information

In [44]:
%%sql

UPDATE
    nashville_housing
SET
    OwnerAddress = SUBSTR(OwnerAddress, INSTR(OwnerAddress, ", ")+2);

 * sqlite:///my_projects.db
56477 rows affected.


[]

- Third, parse owners' city information out of "OwnerAddress" and set values into a new column "SplitOwnerCity"

In [45]:
%%sql 

ALTER TABLE
    nashville_housing
ADD COLUMN
    SplitOwnerCity TEXT(255);

UPDATE
    nashville_housing
SET
    SplitOwnerCity = SUBSTR(OwnerAddress, 1, INSTR(OwnerAddress, ", ")-1);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

- Finally, parse owners' state information out of "OwnerAddress" and set values into a new column "SplitOwnerState"

In [46]:
%%sql

ALTER TABLE
    nashville_housing
ADD COLUMN
    SplitOwnerState TEXT(255);

UPDATE
    nashville_housing
SET
    SplitOwnerState = SUBSTR(OwnerAddress, INSTR(OwnerAddress, ", ")+2);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

#### 3.4 Standardize "Sold as Vacant" column to solely "Yes" or "No"

In [47]:
%%sql

SELECT
    DISTINCT SoldAsVacant,
    COUNT(SoldAsVacant)
FROM
    nashville_housing
GROUP BY
    SoldAsVacant;

 * sqlite:///my_projects.db
Done.


SoldAsVacant,COUNT(SoldAsVacant)
N,399
No,51403
Y,52
Yes,4623


In [48]:
%%sql

UPDATE
    nashville_housing
SET
    SoldAsVacant = (CASE WHEN 
                        SoldAsVacant = "N" THEN "No"
                    WHEN 
                        SoldAsVacant = "Y" THEN "Yes"
                    ELSE 
                        SoldAsVacant
                    END);

 * sqlite:///my_projects.db
56477 rows affected.


[]

#### 3.5 Convert "YearBuilt" column from float to integer

- First, rename "YearBuilt" column to "old_YearBuilt" column for next step

In [49]:
%%sql

ALTER TABLE
    nashville_housing
RENAME COLUMN
    YearBuilt TO old_YearBuilt;

 * sqlite:///my_projects.db
Done.


1


- Then, add a new column "YearBuilt" and set values which cast "old_YearBuilt" column from float to integer

In [51]:
%%sql

ALTER TABLE
    nashville_housing
ADD COLUMN
    YearBuilt INTEGER;

UPDATE
    nashville_housing
SET
    YearBuilt = CAST(old_YearBuilt AS INTEGER);

 * sqlite:///my_projects.db
Done.
56477 rows affected.


[]

### 4. Delete unused columns

In [55]:
%%sql

SELECT sql
FROM sqlite_master
WHERE tbl_name = "nashville_housing";

 * sqlite:///my_projects.db
Done.


sql
"CREATE TABLE ""nashville_housing"" (""UniqueID "" INTEGER, ""ParcelID"" TEXT, ""LandUse"" TEXT, ""PropertyAddress"" TEXT, ""SaleDate"" TIMESTAMP, ""SalePrice"" INTEGER, ""LegalReference"" TEXT, ""SoldAsVacant"" TEXT, ""OwnerName"" TEXT, ""OwnerAddress"" TEXT, ""Acreage"" REAL, ""TaxDistrict"" TEXT, ""LandValue"" REAL, ""BuildingValue"" REAL, ""TotalValue"" REAL, ""old_YearBuilt"" REAL, ""Bedrooms"" REAL, ""FullBath"" REAL, ""HalfBath"" REAL, SplitPropertyAddress TEXT(255), SplitPropertyCity TEXT(255), SplitOwnerAddress TEXT(255), SplitOwnerCity TEXT(255), SplitOwnerState TEXT(255), YearBuilt INTEGER)"
"CREATE INDEX ""ix_nashville_housing_UniqueID ""ON ""nashville_housing"" (""UniqueID "")"


In [56]:
%%sql

PRAGMA foreign_keys=OFF;

CREATE TABLE IF NOT EXISTS nashville_housing_cleaned(
    ParcelID TEXT,
    LandUse TEXT,
    SplitPropertyAddress TEXT,
    SplitPropertyCity TEXT,
    SaleDate TIMESTAMP,
    SalePrice INTEGER,
    LegalReference TEXT,
    SoldAsVacant TEXT,
    OwnerName TEXT,
    SplitOwnerAddress TEXT,
    SplitOwnerCity TEXT,
    SplitOwnerState TEXT,
    Acreage REAL,
    LandValue REAL,
    BuildingValue REAL,
    TotalValue REAL,
    YearBuilt INTEGER,
    Bedrooms REAL,
    FullBath REAL,
    HalfBath REAL
);

INSERT INTO nashville_housing_cleaned

SELECT
    ParcelID,
    LandUse,
    SplitPropertyAddress,
    SplitPropertyCity,
    SaleDate,
    SalePrice,
    LegalReference,
    SoldAsVacant,
    OwnerName,
    SplitOwnerAddress,
    SplitOwnerCity,
    SplitOwnerState,
    Acreage,
    LandValue,
    BuildingValue,
    TotalValue,
    YearBuilt,
    Bedrooms,
    FullBath,
    HalfBath
FROM
    nashville_housing;

DROP TABLE 
    nashville_housing;

ALTER TABLE 
    nashville_housing_cleaned
RENAME TO
    nashville_housing;

PRAGMA foreign_keys=ON;

 * sqlite:///my_projects.db
Done.
Done.
Done.
56477 rows affected.
Done.
Done.
(sqlite3.OperationalError) cannot commit - no transaction is active
[SQL: COMMIT;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
